In [1]:
import numpy as np
import pandas as pd

In [2]:
bigfile_path = r'C:\Users\phsheari\Documents\Van Meyer\510k.xlsx'

In [3]:
USECOLS = ['Asset List Amount','Best Site GU Party Name','GU_ID','SAV IDs','Revised SCMS','Best Site Sales Level 1 Name', 'Best Site Sales Level 2 Name', 'Best Site Sales Level 3 Name','Revised SL1','Revised SL2','Revised SL3']

In [4]:
bigfile = pd.read_excel(bigfile_path, sheet_name='limited', usecols = USECOLS)

In [5]:
bigfile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91383 entries, 0 to 91382
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Asset List Amount             91323 non-null  float64
 1   Best Site GU Party Name       91383 non-null  object 
 2   GU_ID                         81681 non-null  float64
 3   SAV IDs                       81624 non-null  object 
 4   Best Site Sales Level 1 Name  91383 non-null  object 
 5   Best Site Sales Level 2 Name  91383 non-null  object 
 6   Best Site Sales Level 3 Name  91383 non-null  object 
 7   Revised SCMS                  91383 non-null  object 
 8   Revised SL1                   39119 non-null  object 
 9   Revised SL2                   39119 non-null  object 
 10  Revised SL3                   39119 non-null  object 
dtypes: float64(2), object(9)
memory usage: 7.7+ MB


In [6]:
bigfile.rename(columns={'Best Site GU Party Name':'Account',
                        'GU_ID':'GUID',
                        'SAV IDs':'SAVID',
                        'Revised SCMS':'SCMS',
                        'Best Site Sales Level 1 Name':'L1',
                        'Best Site Sales Level 2 Name':'L2',
                        'Best Site Sales Level 3 Name':'L3',
                        'Revised SL1':'RSL1',
                        'Revised SL2':'RSL2',
                        'Revised SL3':'RSL3'}, inplace=True)

In [7]:
bigfile = bigfile.loc[~bigfile['Account'].str.contains('CSC DISCARD', na=False, case=False)]

In [8]:
bigfile = bigfile.loc[bigfile['Asset List Amount'] >= 1000000]

In [9]:
bigfile

,Asset List Amount,Account,GUID,SAVID,L1,L2,L3,SCMS,RSL1,RSL2,RSL3
0,1.332092e+09,HSBC HOLDINGS PLC,80621.0,"203894614,283397087,283429583,203894618,203897...",EMEAR-REGION,EMEAR-UKI,ENTERPRISE_EAW,PREMIER,EMEAR-REGION,EMEAR-UKI,ENTERPRISE_EAW
1,6.535133e+08,EXELON CORPORATION,4748652.0,"283534004,283563767,283715236,283719658,283719...",Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL,KEY,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL
2,4.264073e+07,REGERINGSKANSLIET,4252802.0,"283719659,276408780,283543165,283697418,283703...",EMEAR-REGION,EMEAR-NORTH,COUNTRY_SWEDEN,MAJOR,EMEAR-REGION,EMEAR-NORTH,COUNTRY_SWEDEN
3,3.319456e+10,GOVERNMENT OF THE US,54028.0,"208459096,283713776,203775503,203725337,203720...",Americas,US PS Market Segment,PS FEDERAL AREA,PREMIER,Americas,US PS Market Segment,PS FEDERAL AREA
4,1.292123e+07,HAMILTON HEALTH SCIENCES CORPORATION,2948116.0,"283714559,283717884,203687838",Americas,CANADA,PUBLIC SECTOR CAN,MAJOR,Americas,CANADA,PUBLIC SECTOR CAN
...,...,...,...,...,...,...,...,...,...,...,...
91284,1.096960e+06,CITADEL INVESTMENT GROUP INC,187839.0,"283543165,283549499,283534733,283709127,283696...",EMEAR-REGION,EMEAR-NORTH,SMALL_NORTH,KEY,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL
91298,1.684981e+06,AMERICAN NATURAL GAS LLC,235252751.0,"283714555,283714554,283714561,283714969,203701...",Americas,US COMMERCIAL,SBU_US COMMERCIAL SMALL BUSINESS,MAJOR,NaN,NaN,NaN
91338,3.012362e+06,UNIVERSITY CORPORATION FOR ATMOSPHERIC RESEARCH,37740.0,"283714554,283694954,283505174,203768623,283714...",Americas,US COMMERCIAL,SBU_US COMMERCIAL SMALL BUSINESS,PREMIER,Americas,US PS Market Segment,PS FEDERAL AREA
91381,2.094705e+07,GESTION AUDEM INC,2123673.0,"283715230,283717882,283717883,283714549,203835...",Americas,US COMMERCIAL,SBU_US COMMERCIAL SMALL BUSINESS,KEY,Americas,AMERICAS_SP,ASP_CANADA


In [10]:
def replace_sl(revised_sl, original_sl):
    if pd.isnull(revised_sl) or revised_sl is None:
        return original_sl
    else:
        return revised_sl

In [11]:
bigfile['Sales_Level1'] = bigfile.apply(lambda x: replace_sl(x['RSL1'], x['L1']), axis=1)

In [12]:
bigfile['Sales_Level2'] = bigfile.apply(lambda x: replace_sl(x['RSL2'], x['L2']), axis=1)

In [13]:
bigfile['Sales_Level3'] = bigfile.apply(lambda x: replace_sl(x['RSL3'], x['L3']), axis=1)

In [14]:
bigfile = bigfile[['Account','GUID','SAVID','SCMS','Sales_Level1','Sales_Level2','Sales_Level3']]

In [15]:
#A function to stringify the various possible elements in the dataframe with a float data-type
def stringify_element(m):
    a = pd.NA
    if pd.isnull(m):  # if did is null or nan
        return pd.NA
    elif type(m) == np.float64:
        return str(int(m))
    elif type(m) == float:
        return str(int(m))
    elif type(m) == np.int64:
        return str(m)
    elif type(m) == int:
        return str(m)
    elif type(m) == str:
        try:
            a = m.replace('"','').replace('&',',').replace('(','').replace(')','').replace(':',',').replace(' ',',').replace('\n',',').replace('\\n',',').replace('\\n\\r',',') .replace(',,',',').strip().split(',') 
            return a
        except:
            import pdb; pdb.set_trace()  
    else:
        return m

In [16]:
def convert_nan_to_int(dataframe_cell):
        if pd.isnull(dataframe_cell):
            return pd.NA
        else:
            try:
                return int(dataframe_cell)
            except:
                return pd.NA

In [17]:
def trim_element(e):
    return e.strip()

In [18]:
### 101 records were GUID 9000, CSC DISCARD
### 9702 Records have no GUID or SAV ID
### Van's guidance - remove all records having asset list amount < $1M (amounted to ~3/4ths of the original records 67353/91383)
### separate file:
    # 1 component is name only since 9702 have no ID
    # make a sub dataset containing non-NULL GUIDS or SAVIDs

In [19]:
bf_name_only = bigfile.loc[bigfile['GUID'].isnull() & bigfile['GUID'].isnull()][['Account','GUID','SAVID','SCMS','Sales_Level1','Sales_Level2','Sales_Level3']]

In [20]:
bf_name_only.drop_duplicates(keep='first', inplace=True)

In [21]:
bf_name_only['GUID'] = bf_name_only['GUID'].apply(str)

In [22]:
bf = bigfile.loc[~(bigfile['GUID'].isnull() & bigfile['GUID'].isnull())][['Account','GUID','SAVID','SCMS','Sales_Level1','Sales_Level2','Sales_Level3']]

In [23]:
bf.drop_duplicates(keep='first', inplace=True)

In [24]:
bf

,Account,GUID,SAVID,SCMS,Sales_Level1,Sales_Level2,Sales_Level3
0,HSBC HOLDINGS PLC,80621.0,"203894614,283397087,283429583,203894618,203897...",PREMIER,EMEAR-REGION,EMEAR-UKI,ENTERPRISE_EAW
1,EXELON CORPORATION,4748652.0,"283534004,283563767,283715236,283719658,283719...",KEY,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL
2,REGERINGSKANSLIET,4252802.0,"283719659,276408780,283543165,283697418,283703...",MAJOR,EMEAR-REGION,EMEAR-NORTH,COUNTRY_SWEDEN
3,GOVERNMENT OF THE US,54028.0,"208459096,283713776,203775503,203725337,203720...",PREMIER,Americas,US PS Market Segment,PS FEDERAL AREA
4,HAMILTON HEALTH SCIENCES CORPORATION,2948116.0,"283714559,283717884,203687838",MAJOR,Americas,CANADA,PUBLIC SECTOR CAN
...,...,...,...,...,...,...,...
69895,AFIP,3828794.0,203776409,PREMIER,Americas,LATIN AMERICA,M_C_R
75647,SIG COMBIBLOC GROUP AG,2670377.0,"283388560,283697389,283715230,283703286,283486...",MAJOR,APJC,GREATER_CHINA,CN EAST
83099,BSNL HYDERABAD,164372058.0,"283505489,283534180,283714893,283688130,283716...",MAJOR,APJC,INDIA_AREA,IND_SMALL
89609,"UNIVERSAL ASSET MANAGEMENT, LLC",9163153.0,"283714557,258194802,283714550,283700797,283700...",MAJOR,APJC,GREATER_CHINA,GC_SMALL


In [25]:
bf['GUID'] = bf['GUID'].astype(int)

In [26]:
bf1 = bf[['GUID','SAVID']]

In [27]:
bf1['SAVID'] = bf1['SAVID'].apply(stringify_element)

<ipython-input-27-8adf9081504b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bf1['SAVID'] = bf1['SAVID'].apply(stringify_element)


In [28]:
bf1.set_index('GUID', inplace=True)

In [29]:
bf1

,SAVID
GUID,
80621,"[203894614, 283397087, 283429583, 203894618, 2..."
4748652,"[283534004, 283563767, 283715236, 283719658, 2..."
4252802,"[283719659, 276408780, 283543165, 283697418, 2..."
54028,"[208459096, 283713776, 203775503, 203725337, 2..."
2948116,"[283714559, 283717884, 203687838]"
...,...
3828794,[203776409]
2670377,"[283388560, 283697389, 283715230, 283703286, 2..."
164372058,"[283505489, 283534180, 283714893, 283688130, 2..."


In [30]:
bfex = bf1.explode('SAVID')

In [31]:
bfex.reset_index(inplace=True)

In [32]:
bfex.drop_duplicates(keep='first', inplace=True)

In [33]:
bf_cleaned = bfex.merge(bf, how='left', left_on = 'GUID', right_on = 'GUID')[['GUID','SAVID_x','Account','SCMS','Sales_Level1','Sales_Level2','Sales_Level3']]

In [34]:
bf_cleaned.rename(columns={'SAVID_x':'SAVID'}, inplace=True)

In [35]:
bf_cleaned['SAVID'] = bf_cleaned['SAVID'].astype(str)

In [36]:
bf_cleaned['SAVID'] = bf_cleaned['SAVID'].apply(trim_element)

In [37]:
bf_cleaned['GUID'] = bf_cleaned['GUID'].astype(str)

In [38]:
bf_cleaned['GUID'] = bf_cleaned['GUID'].apply(trim_element)

In [39]:
bf_guid_only = bf_cleaned.loc[bf_cleaned['SAVID'].isnull()]

In [40]:
bf_guid_only

,GUID,SAVID,Account,SCMS,Sales_Level1,Sales_Level2,Sales_Level3


In [41]:
bf_guids_sav = bf_cleaned.loc[bf_cleaned['SAVID'].notnull()]

In [42]:
bf_name_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165 entries, 72 to 90550
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Account       165 non-null    object
 1   GUID          165 non-null    object
 2   SAVID         0 non-null      object
 3   SCMS          165 non-null    object
 4   Sales_Level1  165 non-null    object
 5   Sales_Level2  165 non-null    object
 6   Sales_Level3  165 non-null    object
dtypes: object(7)
memory usage: 10.3+ KB


In [43]:
bf_guid_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   GUID          0 non-null      object
 1   SAVID         0 non-null      object
 2   Account       0 non-null      object
 3   SCMS          0 non-null      object
 4   Sales_Level1  0 non-null      object
 5   Sales_Level2  0 non-null      object
 6   Sales_Level3  0 non-null      object
dtypes: object(7)
memory usage: 0.0+ bytes


In [44]:
bf_guids_sav.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133019 entries, 0 to 133018
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   GUID          133019 non-null  object
 1   SAVID         133019 non-null  object
 2   Account       133019 non-null  object
 3   SCMS          133019 non-null  object
 4   Sales_Level1  133019 non-null  object
 5   Sales_Level2  133019 non-null  object
 6   Sales_Level3  133019 non-null  object
dtypes: object(7)
memory usage: 8.1+ MB


In [45]:
bf_guids_sav

,GUID,SAVID,Account,SCMS,Sales_Level1,Sales_Level2,Sales_Level3
0,80621,203894614,HSBC HOLDINGS PLC,PREMIER,EMEAR-REGION,EMEAR-UKI,ENTERPRISE_EAW
1,80621,283397087,HSBC HOLDINGS PLC,PREMIER,EMEAR-REGION,EMEAR-UKI,ENTERPRISE_EAW
2,80621,283429583,HSBC HOLDINGS PLC,PREMIER,EMEAR-REGION,EMEAR-UKI,ENTERPRISE_EAW
3,80621,203894618,HSBC HOLDINGS PLC,PREMIER,EMEAR-REGION,EMEAR-UKI,ENTERPRISE_EAW
4,80621,203897042,HSBC HOLDINGS PLC,PREMIER,EMEAR-REGION,EMEAR-UKI,ENTERPRISE_EAW
...,...,...,...,...,...,...,...
133014,235252751,283714561,AMERICAN NATURAL GAS LLC,MAJOR,Americas,US COMMERCIAL,SBU_US COMMERCIAL SMALL BUSINESS
133015,235252751,283714969,AMERICAN NATURAL GAS LLC,MAJOR,Americas,US COMMERCIAL,SBU_US COMMERCIAL SMALL BUSINESS
133016,235252751,203701778,AMERICAN NATURAL GAS LLC,MAJOR,Americas,US COMMERCIAL,SBU_US COMMERCIAL SMALL BUSINESS
133017,235252751,283714558,AMERICAN NATURAL GAS LLC,MAJOR,Americas,US COMMERCIAL,SBU_US COMMERCIAL SMALL BUSINESS


In [46]:
### We have 3 datasets for searching from the BIGFILE:
# bf_name_only
# bf_guid_only
# bf_guids_savs

In [47]:
trkrfile = r'C:\Users\phsheari\Documents\SFDC Data\PIPELINE UPDATE\SFDC_Trackerfile.pkl'

In [48]:
ro = pd.read_pickle(trkrfile)

In [49]:
ro.rename(columns={'SAV ID':'SAVID',
                   'Customer Name':'Account'}, inplace=True)

In [50]:
colist = list(ro.columns)

In [51]:
# colist

In [52]:
ro[['Request ID','Account','GUID','SAVID','HashedReadout', 'Forecast Status','Request FY','Readout FY','Report FY']]

,Request ID,Account,GUID,SAVID,HashedReadout,Forecast Status,Request FY,Readout FY,Report FY
0,201,BEAUMONT HOSPITAL,NaN,NaN,<NA>,,2019,<NA>,2019
1,1,UNITED SHORE FINANCIAL SERVICES LLC,NaN,NaN,<NA>,,2019,2020,2020
2,2,BALLAD HEALTH,NaN,NaN,<NA>,,2019,2020,2020
3,3,ABM INDUSTRIES,203764509,203764509,<NA>,,2019,2020,2020
4,4,GRAPHIC PACKAGING INTERNATIONAL INC,NaN,NaN,<NA>,,2019,2020,2020
...,...,...,...,...,...,...,...,...,...
40452,116375,FAURECIA,2127906,NaN,<NA>,,2022,<NA>,2022
40453,116376,HURLEY HOSPITAL,35289,NaN,<NA>,,2022,<NA>,2022
40454,116377,AUTO CLUB GROUP,11878,NaN,<NA>,,2022,<NA>,2022
40455,116378,MEMBER DRIVEN TECHNOLOGIES,9706672,NaN,<NA>,,2022,<NA>,2022


In [53]:
ro_names_only = ro.loc[(ro['GUID'].isnull()) & (ro['SAVID'].isnull())][['Request ID','Account','GUID','SAVID']]

In [54]:
ro_names_only

,Request ID,Account,GUID,SAVID
0,201,BEAUMONT HOSPITAL,NaN,NaN
1,1,UNITED SHORE FINANCIAL SERVICES LLC,NaN,NaN
2,2,BALLAD HEALTH,NaN,NaN
4,4,GRAPHIC PACKAGING INTERNATIONAL INC,NaN,NaN
5,5,JOHN DEERE,NaN,NaN
...,...,...,...,...
18383,20107,CANADIAN TIRE,NaN,NaN
19523,20106,CANADIAN NATIONAL RAILWAY,NaN,NaN
20982,21505,AMPHIA ZIEKENHUIS,NaN,NaN
20983,21506,CANADIAN TIRE,NaN,NaN


In [55]:
roguids_only = ro.loc[(ro['GUID'].notnull()) & (ro['SAVID'].isnull())][['Request ID','GUID','SAVID']]

In [56]:
roguids_only['GUID'] = roguids_only['GUID'].apply(trim_element)

In [57]:
roguids_only

,Request ID,GUID,SAVID
7,7,32715,NaN
8,8,27581,NaN
9,10,32715,NaN
14,16,3392386,NaN
20,22,3403057,NaN
...,...,...,...
40452,116375,2127906,NaN
40453,116376,35289,NaN
40454,116377,11878,NaN
40455,116378,9706672,NaN


In [58]:
roguids_savs = ro.loc[(ro['GUID'].notnull()) & (ro['SAVID'].notnull())][['Request ID','GUID','SAVID']]

In [59]:
roguids_savs.set_index(['Request ID','GUID'], inplace=True)

In [60]:
roguids_savs['SAVID'] = roguids_savs['SAVID'].apply(stringify_element)

In [61]:
roguids_savs = roguids_savs.explode('SAVID')

In [62]:
roguids_savs.reset_index()

,Request ID,GUID,SAVID
0,3,203764509,203764509
1,6,32715,199060442
2,49,6605477,203725064
3,63,1223425,203854333
4,69,204551435,203774836
...,...,...,...
30958,116364,4266634,283443837
30959,116365,46705,203779018
30960,116369,"4518915,47733",203837302
30961,116370,42629,203733315


In [63]:
def convert_listitem_to_int(m):
    if isinstance(m, list):
        for e in m:
            return str(e)
    else:
        return str(e)

In [64]:
roguids_savs['SAVL'] = roguids_savs['SAVID'].str.len()

In [65]:
roguids_savs

,,SAVID,SAVL
Request ID,GUID,,
3,203764509,203764509,9
6,32715,199060442,9
49,6605477,203725064,9
63,1223425,203854333,9
69,204551435,203774836,9
...,...,...,...
116364,4266634,283443837,9
116365,46705,203779018,9
116369,"4518915,47733",203837302,9


In [66]:
roguids_savs.reset_index(inplace=True)

In [67]:
roguids_savs.loc[roguids_savs['SAVL'] > 10]

,Request ID,GUID,SAVID,SAVL
2144,2782,2123742,2.83542488283524E+26,20
19824,100011,26382,2.03769731253271E+26,20
19825,100012,26382,2.03769731253271E+26,20
19826,100013,26382,2.03769731253271E+26,20
19827,100014,26382,2.03769731253271E+26,20
19839,100010,91375,2.03792071203734E+35,20


In [68]:
roguids_savs.drop(roguids_savs.loc[roguids_savs['SAVL'] > 10].index, inplace=True)

In [69]:
# roguids_savs_wi = roguids_savs.loc[roguids_savs['SAVL'] > 10][['Request ID','GUID','SAVID']]

In [70]:
# roguids_savs_wi

In [71]:
# roguids_savs_wi.set_index('Request ID','GUID', inplace=True)

In [72]:
# roguids_savs_wi = roguids_savs_wi.explode('SAVID')

In [73]:
# roguids_savs_wi.reset_index(inplace=True)

In [74]:
# roguids_savs_wi

In [75]:
roguids_savs = roguids_savs[['Request ID','GUID','SAVID']]

In [76]:
# roguids_savs = pd.concat([roguids_savs, roguids_savs_wi])

In [77]:
roguids_savs

,Request ID,GUID,SAVID
0,3,203764509,203764509
1,6,32715,199060442
2,49,6605477,203725064
3,63,1223425,203854333
4,69,204551435,203774836
...,...,...,...
30958,116364,4266634,283443837
30959,116365,46705,203779018
30960,116369,"4518915,47733",203837302
30961,116370,42629,203733315


In [78]:
# roguids_savs.set_index(['Request ID','SAVID'], inplace=True)

In [79]:
roguids_savs['GUID'] = roguids_savs['GUID'].apply(stringify_element)

<ipython-input-79-065fbe7bfe35>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  roguids_savs['GUID'] = roguids_savs['GUID'].apply(stringify_element)


In [80]:
roguids_savs = roguids_savs.explode('GUID')

In [81]:
roguids_savs.reset_index(inplace=True)

In [82]:
roguids_savs = roguids_savs[['Request ID','GUID','SAVID']]

In [83]:
roguids_savs.drop(roguids_savs.loc[roguids_savs['GUID'].str.contains('and', na=False, case=False)].index, inplace=True)

In [84]:
roguids_savs['GUIDL'] = roguids_savs['GUID'].str.len()

In [85]:
roguids_savs.loc[(roguids_savs['GUID'].str.contains('54028', na=False, case=False)) & (roguids_savs['GUIDL'] > 5)]

,Request ID,GUID,SAVID,GUIDL
5800,1843,157554028,203866404,9
43313,5725,157554028,203866404,9
120161,11693,154028,283505149,6
121551,10090,157554028,203866404,9


In [86]:
roguids_savs.loc[(roguids_savs['GUID'].str.startswith('54028'))]

,Request ID,GUID,SAVID,GUIDL
829,1159,54028,203729604,5
942,1079,54028,203800970,5
996,1421,54028,203719538,5
1019,1445,54028,203684841,5
1070,1499,54028,208459096,5
...,...,...,...,...
150367,113528,54028,212625035,5
150368,113557,54028,203836621,5
150493,113689,54028,203686945,5
150788,114057,54028,203686902,5


In [87]:
roguids_savs['GUID'] = roguids_savs['GUID'].apply(trim_element)

In [88]:
roguids_savs['SAVID'] = roguids_savs['SAVID'].apply(trim_element)

In [89]:
roguids_savs.drop(columns='GUIDL', inplace=True)

In [90]:
roguids_savs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152997 entries, 0 to 153002
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Request ID  152997 non-null  object
 1   GUID        152997 non-null  object
 2   SAVID       152997 non-null  object
dtypes: object(3)
memory usage: 4.7+ MB


In [91]:
roguids_savs

,Request ID,GUID,SAVID
0,3,203764509,203764509
1,6,32715,199060442
2,49,6605477,203725064
3,63,1223425,203854333
4,69,204551435,203774836
...,...,...,...
152998,116365,46705,203779018
152999,116369,4518915,203837302
153000,116369,47733,203837302
153001,116370,42629,203733315


In [143]:
comb_guids_sav = roguids_savs.merge(bf_guids_sav, how='left', left_on = ['SAVID'], right_on = ['SAVID'])

In [144]:
comb_guids_sav = comb_guids_sav.loc[comb_guids_sav['Account'].notnull()]

In [145]:
comb_guids_sav

,Request ID,GUID_x,SAVID,GUID_y,Account,SCMS,Sales_Level1,Sales_Level2,Sales_Level3
8,370,175670,255147400,10492,INTERNATIONAL BUSINESS MACHINES CORPORATION,PREMIER,Americas,GLOBAL ENTERPRISE SEGMENT,GES PREMIER
9,370,175670,255147400,145630024,ACE LTD,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES EAST
10,374,9639216,203771654,9639216,INCOMM HOLDINGS INC,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES SOUTH
18,527,20816,203844004,2639582,CENTURION MEDICAL PRODUCTS CORPORATION,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL
19,527,20816,203844004,20816,MEDLINE INDUSTRIES INC,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL
...,...,...,...,...,...,...,...,...,...
816260,116351,45471,203812869,4051570,AGNS,PREMIER,Americas,AMERICAS_SP,ASP CSP
816261,116362,4260639,203896025,4260639,SCHWEIZERISCHE EIDGENOSSENSCHAFT,MAJOR,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND
816262,116362,4260639,203896025,147358640,AXP HOLDING COMPANY BV,MAJOR,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND
816265,116365,46705,203779018,25752,WMATA,MAJOR,Americas,US PS Market Segment,SLED-East Area


In [146]:
roguids_only['GUID'] = roguids_only['GUID'].apply(trim_element)

In [147]:
comb_guids_only = roguids_only.merge(bf_guids_sav, how='left', left_on='GUID', right_on='GUID')

In [148]:
comb_guids_only = comb_guids_only.loc[comb_guids_only['Account'].notnull()]

In [149]:
comb_guids_only.drop_duplicates(keep='first', inplace=True)

In [150]:
comb_guids_only

,Request ID,GUID,SAVID_x,SAVID_y,Account,SCMS,Sales_Level1,Sales_Level2,Sales_Level3
0,7,32715,NaN,261951730,BLUE CROSS & BLUE SHIELD ASSOCIATION,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL
1,8,27581,NaN,203738203,CALIFORNIA PHYSICIANS' SERVICE,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES WEST
2,10,32715,NaN,261951730,BLUE CROSS & BLUE SHIELD ASSOCIATION,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL
6,40,35596,NaN,203809838,CITY OF VIRGINIA BEACH,MAJOR,Americas,US PS Market Segment,SLED-East Area
7,40,35596,NaN,283717884,CITY OF VIRGINIA BEACH,MAJOR,Americas,US PS Market Segment,SLED-East Area
...,...,...,...,...,...,...,...,...,...
436330,116375,2127906,NaN,283713746,PEUGEOT SA,KEY,EMEAR-REGION,EMEAR-SOUTH,COUNTRY_FRANCE
436331,116375,2127906,NaN,283557771,PEUGEOT SA,KEY,EMEAR-REGION,EMEAR-SOUTH,COUNTRY_FRANCE
436332,116375,2127906,NaN,283703172,PEUGEOT SA,KEY,EMEAR-REGION,EMEAR-SOUTH,COUNTRY_FRANCE
436333,116375,2127906,NaN,203897748,PEUGEOT SA,KEY,EMEAR-REGION,EMEAR-SOUTH,COUNTRY_FRANCE


In [151]:
ro_names_only['Account'] = ro_names_only['Account'].apply(trim_element)

In [152]:
bf_name_only['Account'] = bf_name_only['Account'].apply(trim_element)

In [153]:
comb_name_only = ro_names_only.merge(bf_name_only, how='left', on='Account')

In [154]:
comb_name_only = comb_name_only.loc[(comb_name_only['GUID_y'].notnull()) | (comb_name_only['SAVID_y'].notnull())]

In [155]:
comb_guids_sav

,Request ID,GUID_x,SAVID,GUID_y,Account,SCMS,Sales_Level1,Sales_Level2,Sales_Level3
8,370,175670,255147400,10492,INTERNATIONAL BUSINESS MACHINES CORPORATION,PREMIER,Americas,GLOBAL ENTERPRISE SEGMENT,GES PREMIER
9,370,175670,255147400,145630024,ACE LTD,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES EAST
10,374,9639216,203771654,9639216,INCOMM HOLDINGS INC,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES SOUTH
18,527,20816,203844004,2639582,CENTURION MEDICAL PRODUCTS CORPORATION,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL
19,527,20816,203844004,20816,MEDLINE INDUSTRIES INC,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL
...,...,...,...,...,...,...,...,...,...
816260,116351,45471,203812869,4051570,AGNS,PREMIER,Americas,AMERICAS_SP,ASP CSP
816261,116362,4260639,203896025,4260639,SCHWEIZERISCHE EIDGENOSSENSCHAFT,MAJOR,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND
816262,116362,4260639,203896025,147358640,AXP HOLDING COMPANY BV,MAJOR,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND
816265,116365,46705,203779018,25752,WMATA,MAJOR,Americas,US PS Market Segment,SLED-East Area


In [156]:
comb_guids_only.drop(columns='SAVID_x', inplace=True)

In [157]:
comb_guids_only.rename(columns={'SAVID_y':'SAVID'}, inplace=True)

In [158]:
comb_name_only.drop(columns=['GUID_y','SAVID_y'], inplace=True)

In [159]:
comb_name_only.rename(columns={'GUID_x':'GUID', 'SAVID_x':'SAVID'}, inplace=True)

In [160]:
comb_name_only

,Request ID,Account,GUID,SAVID,SCMS,Sales_Level1,Sales_Level2,Sales_Level3
328,15259,KLINIKUM DER JOHANN-WOLFGANG-GOETHE UNIVERSITA...,NaN,NaN,KEY,EMEAR-REGION,EMEAR_GERMANY,PUBLIC_SECTOR_DEU


In [161]:
fullset = pd.concat([comb_name_only, comb_guids_only, comb_guids_sav])

In [162]:
fullset

,Request ID,Account,GUID,SAVID,SCMS,Sales_Level1,Sales_Level2,Sales_Level3,GUID_x,GUID_y
328,15259,KLINIKUM DER JOHANN-WOLFGANG-GOETHE UNIVERSITA...,NaN,NaN,KEY,EMEAR-REGION,EMEAR_GERMANY,PUBLIC_SECTOR_DEU,NaN,NaN
0,7,BLUE CROSS & BLUE SHIELD ASSOCIATION,32715,261951730,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL,NaN,NaN
1,8,CALIFORNIA PHYSICIANS' SERVICE,27581,203738203,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES WEST,NaN,NaN
2,10,BLUE CROSS & BLUE SHIELD ASSOCIATION,32715,261951730,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL,NaN,NaN
6,40,CITY OF VIRGINIA BEACH,35596,203809838,MAJOR,Americas,US PS Market Segment,SLED-East Area,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
816260,116351,AGNS,NaN,203812869,PREMIER,Americas,AMERICAS_SP,ASP CSP,45471,4051570
816261,116362,SCHWEIZERISCHE EIDGENOSSENSCHAFT,NaN,203896025,MAJOR,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,4260639,4260639
816262,116362,AXP HOLDING COMPANY BV,NaN,203896025,MAJOR,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,4260639,147358640
816265,116365,WMATA,NaN,203779018,MAJOR,Americas,US PS Market Segment,SLED-East Area,46705,25752


In [163]:
fullset.drop_duplicates(keep='first', inplace=True)

In [164]:
fullset = fullset.merge(ro[['Request ID','HashedReadout','Forecast Status','Request FY','Readout FY', 'Report FY']], how='left', on='Request ID')

In [166]:
fullset.rename(columns={'GUID_x':'GUID_Trcker','GUID_y': 'GUID_Reference'}, inplace=True)

In [167]:
fullset

,Request ID,Account,GUID,SAVID,SCMS,Sales_Level1,Sales_Level2,Sales_Level3,GUID_Trcker,GUID_Reference,HashedReadout,Forecast Status,Request FY,Readout FY,Report FY
0,15259,KLINIKUM DER JOHANN-WOLFGANG-GOETHE UNIVERSITA...,NaN,NaN,KEY,EMEAR-REGION,EMEAR_GERMANY,PUBLIC_SECTOR_DEU,NaN,NaN,<NA>,,2021,<NA>,2021
1,7,BLUE CROSS & BLUE SHIELD ASSOCIATION,32715,261951730,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL,NaN,NaN,<NA>,,2019,2020,2020
2,8,CALIFORNIA PHYSICIANS' SERVICE,27581,203738203,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES WEST,NaN,NaN,<NA>,,2019,2020,2020
3,10,BLUE CROSS & BLUE SHIELD ASSOCIATION,32715,261951730,MAJOR,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL,NaN,NaN,<NA>,,2019,<NA>,2019
4,40,CITY OF VIRGINIA BEACH,35596,203809838,MAJOR,Americas,US PS Market Segment,SLED-East Area,NaN,NaN,<NA>,,2019,<NA>,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1178313,116351,AGNS,NaN,203812869,PREMIER,Americas,AMERICAS_SP,ASP CSP,45471,4051570,<NA>,,2022,<NA>,2022
1178314,116362,SCHWEIZERISCHE EIDGENOSSENSCHAFT,NaN,203896025,MAJOR,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,4260639,4260639,<NA>,,2022,<NA>,2022
1178315,116362,AXP HOLDING COMPANY BV,NaN,203896025,MAJOR,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,4260639,147358640,<NA>,,2022,<NA>,2022
1178316,116365,WMATA,NaN,203779018,MAJOR,Americas,US PS Market Segment,SLED-East Area,46705,25752,<NA>,,2022,<NA>,2022


In [169]:
fullset.to_csv(r'C:\Users\phsheari\Documents\Van Meyer\FULLSET.txt', sep='\t', index=False)

In [135]:
outermatched = bigfile.merge(fullset[['Request ID','Account']], how = 'left', on='Account')

In [136]:
outermatched

,Account,GUID,SAVID,SCMS,Sales_Level1,Sales_Level2,Sales_Level3,Request ID
0,HSBC HOLDINGS PLC,80621.0,"203894614,283397087,283429583,203894618,203897...",PREMIER,EMEAR-REGION,EMEAR-UKI,ENTERPRISE_EAW,4547
1,HSBC HOLDINGS PLC,80621.0,"203894614,283397087,283429583,203894618,203897...",PREMIER,EMEAR-REGION,EMEAR-UKI,ENTERPRISE_EAW,4547
2,HSBC HOLDINGS PLC,80621.0,"203894614,283397087,283429583,203894618,203897...",PREMIER,EMEAR-REGION,EMEAR-UKI,ENTERPRISE_EAW,4547
3,HSBC HOLDINGS PLC,80621.0,"203894614,283397087,283429583,203894618,203897...",PREMIER,EMEAR-REGION,EMEAR-UKI,ENTERPRISE_EAW,4547
4,HSBC HOLDINGS PLC,80621.0,"203894614,283397087,283429583,203894618,203897...",PREMIER,EMEAR-REGION,EMEAR-UKI,ENTERPRISE_EAW,4547
...,...,...,...,...,...,...,...,...
7918228,GESTION AUDEM INC,2123673.0,"283715230,283717882,283717883,283714549,203835...",KEY,Americas,AMERICAS_SP,ASP_CANADA,17278
7918229,GESTION AUDEM INC,2123673.0,"283715230,283717882,283717883,283714549,203835...",KEY,Americas,AMERICAS_SP,ASP_CANADA,19464
7918230,GESTION AUDEM INC,2123673.0,"283715230,283717882,283717883,283714549,203835...",KEY,Americas,AMERICAS_SP,ASP_CANADA,112643
7918231,GESTION AUDEM INC,2123673.0,"283715230,283717882,283717883,283714549,203835...",KEY,Americas,AMERICAS_SP,ASP_CANADA,114687


In [137]:
notmatched = outermatched.loc[outermatched['Request ID'].isnull()]

In [138]:
notmatched.drop_duplicates(keep='first', inplace=True)

<ipython-input-138-b41d2ea033e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notmatched.drop_duplicates(keep='first', inplace=True)


In [141]:
notmatched = notmatched[['Account', 'GUID', 'SAVID', 'SCMS', 'Sales_Level1', 'Sales_Level2', 'Sales_Level3']]

In [142]:
notmatched.to_excel(r'C:\Users\phsheari\Documents\Van Meyer\NOTMATCHED.xlsx', index=False)